In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
logList=glob.glob('logs/Monitoring_logs/*.txt')

In [3]:
def to_config(line,access):
    return eval(line.replace(" ".join(line.split(' ')[:4]),'').replace(f'%s Mismatches'%access,"{'%s'"%access)+"}")

def get_registers(idict):
    parameters = []
    for access,accessDict in idict.items():
        if 'ASIC' in accessDict.keys():
            accessDict = accessDict['ASIC']
        for block,blockDict in accessDict.items():
            for param, paramDict in blockDict.items():
                parameters.append(param)
    return parameters

RW_hardreset = {}
RO_hardreset = {}
with open('logs/Monitoring_logs/log_March_12_InBeam_ForcedReset.txt', 'r') as f:
    for line in f:
        if 'RW Mismatches' in line and not RW_hardreset:
            RW_hardreset = to_config(line,'RW')
        if 'RO Mismatches' in line and not RW_hardreset:
            RO_hardreset = to_config(line,'RO')
        
to_read = [
    'n_OOT_errors',
    'n_IT_errors',
    'n_RW_mismatches',
    'n_RO_mismatches',
    'hardresets',
    'RW_mismatches',
    'RO_mismatches',
]

logList=glob.glob('logs/Monitoring_logs/*.txt')
monitoring_logs = {}

for log in logList:
    file1 = open(log, 'r')
    lines = file1.readlines()
    key = ""
    for i,line in enumerate(lines):
        try:
            if 'Stopping comparisons' in line and 'PULSE IS COMING' in lines[i-1]:
                key = ' '.join(line.split(' ')[:2])
                monitoring_logs[key] = dict.fromkeys(to_read)
            elif 'Stopping OOT' in line and 'Error counter' in lines[i+1]:
                monitoring_logs[key]['n_OOT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
            elif 'Stopping comparisons' in line and 'Error counter' in lines[i+1]:
                monitoring_logs[key]['n_IT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
            elif 'RW Mismatches' in line:
                config = to_config(line,'RW')
                monitoring_logs[key]['RW_mismatches'] = get_registers(config)
                monitoring_logs[key]['n_RW_mismatches'] = len(monitoring_logs[key]['RW_mismatches'])
                if config==RW_hardreset:
                    monitoring_logs[key]['hardresets'] = 1
                else:
                    monitoring_logs[key]['hardresets'] = None
            elif 'RO Mismatches' in line:
                config = to_config(line,'RO')
                monitoring_logs[key]['RO_mismatches'] = get_registers(config)
                monitoring_logs[key]['n_RO_mismatches'] = len(monitoring_logs[key]['RO_mismatches'])
        except:
            print('Error parsing ',log)
            break

config = {
    'Time': [],
    'HardReset': [],
    'nRW': [],
    'nRO': [],
    'nOOT': [],
    'nIT': [],
}
for key,item in monitoring_logs.items():
    config['Time'].append(key)
    config['HardReset'].append(item['hardresets'] if item['hardresets'] else 0)
    config['nRW'].append(item['n_RW_mismatches'] if item['n_RW_mismatches'] else 0)
    config['nRO'].append(item['n_RO_mismatches'] if item['n_RO_mismatches'] else 0)
    config['nOOT'].append(item['n_OOT_errors'] if item['n_OOT_errors'] else 0)
    config['nIT'].append(item['n_IT_errors'] if item['n_IT_errors'] else 0)

df=pd.DataFrame.from_dict(config)
df.set_index('Time',inplace=True)
df.sort_index(inplace=True)

Error parsing  logs/Monitoring_logs/log_March_11_test.txt


In [6]:
df

,HardReset,nRW,nRO,nOOT,nIT
Time,,,,,
03-11-22 14:34:33,0,0,0,0,0
03-11-22 14:35:33,0,0,0,0,0
03-11-22 14:36:33,0,0,0,0,0
03-11-22 14:39:33,0,0,0,0,0
03-11-22 14:40:33,0,0,0,0,0
...,...,...,...,...,...
03-17-22 17:15:59,0,0,0,0,0
03-17-22 17:16:59,0,0,0,1,0
03-17-22 17:17:59,0,0,0,0,0


In [ ]:
df.to_csv('logs/MonitoringInfo.csv',index=False)

In [4]:
for key in df.index:
    if '03-11-22 17:38' in key:
        print(key)


03-11-22 17:38:56


In [5]:
df_error=pd.read_csv('logs/ErrorInfo.csv')
df_error.set_index('Time',inplace=True)
df_error

,fName_ASIC,fName_Emulator,fName_Input,RelPosition,MotionTable,Errors,logFile
Time,,,,,,,
2022-03-11 17:38:56,logs/DAQ_logs/lc-ASIC_11mar_173856.csv,logs/DAQ_logs/lc-emulator_11mar_173856.csv,logs/DAQ_logs/lc-input_11mar_173856.csv,25,269,92955406,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:39:26,logs/DAQ_logs/lc-ASIC_oot_11mar_173926.csv,logs/DAQ_logs/lc-emulator_oot_11mar_173926.csv,logs/DAQ_logs/lc-input_oot_11mar_173926.csv,25,269,360536469,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:39:56,logs/DAQ_logs/lc-ASIC_11mar_173956.csv,logs/DAQ_logs/lc-emulator_11mar_173956.csv,logs/DAQ_logs/lc-input_11mar_173956.csv,25,269,400565817,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:40:26,logs/DAQ_logs/lc-ASIC_oot_11mar_174026.csv,logs/DAQ_logs/lc-emulator_oot_11mar_174026.csv,logs/DAQ_logs/lc-input_oot_11mar_174026.csv,25,269,400618586,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:40:57,logs/DAQ_logs/lc-ASIC_11mar_174057.csv,logs/DAQ_logs/lc-emulator_11mar_174057.csv,logs/DAQ_logs/lc-input_11mar_174057.csv,25,269,400636805,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
...,...,...,...,...,...,...,...
2022-03-17 17:03:56,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_17mar_17...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_17ma...,logs/DAQ_logs/lc-input_tsalgo_randdata_17mar_1...,25,269,28,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
2022-03-17 17:04:27,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_oot_17ma...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_oot_...,logs/DAQ_logs/lc-input_tsalgo_randdata_oot_17m...,25,269,3,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
2022-03-17 17:04:57,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_17mar_17...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_17ma...,logs/DAQ_logs/lc-input_tsalgo_randdata_17mar_1...,25,269,34,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
